In [ ]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:

def load_df(csv_path="./Data/train_v2.csv", nrows=None):
    
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

#print(os.listdir("./Data"))

In [ ]:
df_train = load_df("./Data/train_v2.csv")

In [ ]:
import ast
# for customDimensions and hits columns
def parse_special_col(raw_str):
    lst = ast.literal_eval(raw_str)
    if isinstance(lst, list) and lst:
        return pd.Series(lst[0])
    else:
        return pd.Series({})

In [ ]:
df_train.columns

In [ ]:
columns = df_train.columns
non_empty = []
empty = []

for column in columns:
    count = len(df_train[column].unique())
    if count >1:
        non_empty.append(column)
    else:
        empty.append(column)

In [ ]:
df_train.drop(empty,axis=1,inplace=True)
df_train.dropna(subset = ['totals.transactionRevenue'],inplace = True)
df_train.drop(['customDimensions','totals.bounces'],axis=1,inplace=True)

In [ ]:
hits_df = df_train["hits"].apply(parse_special_col)
hits_df.columns = [f'hits_{x}' for x in hits_df.columns]
df = pd.concat([df_train, hits_df], axis=1)
df.drop('hits', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df_train.fillna(0)

In [ ]:
df_train.to_csv("./Data/train-flattened.csv", index=False)

# Repeat the same steps for test data set

In [ ]:
df_test = load_df("./Data/test_v2.csv")

In [ ]:
df_test.columns

In [ ]:
columns = df_test.columns
non_empty = []
empty = []

for column in columns:
    count = len(df_test[column].unique())
    if count >1:
        non_empty.append(column)
    else:
        empty.append(column)

In [ ]:
df_test.drop(empty,axis=1,inplace=True)
#df_test.dropna(subset = ['totals.transactionRevenue'],inplace = True)
df_test.drop(['hits','customDimensions','totals.bounces'],axis=1,inplace=True)

In [ ]:
df_test.shape

In [ ]:
df_test.fillna(0)

In [ ]:
df_test.to_csv("./Data/test-flattened.csv", index=False)